In [165]:
import pandas as pd

In [166]:
def tablematch(
    df_tablematch_input, # Tablematch Rules
    df_main_table_input, # Input Table
    tm_result, # Result Column
    tm_active = None, # Name of Boolean Column to filter aktive Rules
    tm_order = None, # Name of Int Column to sort Rules
    tm_category = None, # String Column to filter Categories
    input_category = None, # Input which Category to use
    tm_comment = None, # Comment of Rule
    case_sensitive = True, # Case Sensitive Data, otherwise everything will be upper case. Carefull with inplace
    inplace = False, # Replace existing Dataframe or add Result Column
    rule_delimiter = ',', # Delimiter for the rules
    ): 

    df_main_table = df_main_table_input.copy()

    df_tablematch = df_tablematch_input.copy()

    # Nur aktive Regeln behalten
    if tm_active is not None:
        df_tablematch = df_tablematch.loc[df_tablematch[tm_active] == True,:].copy()
        df_tablematch.drop(columns = [tm_active], inplace = True)

    # Nach Category Filtern
    if input_category is not None and tm_category is not None:
        df_tablematch = df_tablematch.loc[df_tablematch[tm_category] == input_category,:].copy()
    if tm_category is not None:
        df_tablematch.drop(columns = [tm_category], inplace = True)

    # Sortieren nach tm_order
    if tm_order is not None:
        df_tablematch.sort_values(by = tm_order, inplace = True)
        df_tablematch.drop(columns = [tm_order], inplace = True)

    # Drop Comment
    if tm_comment is not None:
        df_tablematch.drop(columns = [tm_comment], inplace = True)

    # Alle wichtigen Spalten sammeln
    tablematch_columns = df_tablematch.columns.to_list()
    tablematch_columns.remove(tm_result)

    # Überschneidene Spalten finden
    main_table_columns = df_main_table.columns.to_list()
    column_inner = list(set(main_table_columns) & set(tablematch_columns))

    # Check ob result schon existiert
    if tm_result in main_table_columns:
        print(tm_result + ' ist bereits im Datenset vorhanden! Tablematch abgebrochen') # TODO Müsste eigentlich hier abbrechen
        return df_main_table_input
    else:
        # Dict fürs Mapping von Boolean
        boolean_to_string = {True: 'True', False: 'False'}

        mask_tablematch = df_tablematch.applymap(type) != bool
        # df_tablematch = df_tablematch.where(mask_tablematch, df_tablematch.replace(boolean_to_string)) # TODO

        # Jede Spalte durchgehen und bei Bedarf verändern
        for column in tablematch_columns:
            df_tablematch = df_tablematch.where(mask_tablematch[column], df_tablematch[column].map(boolean_to_string), axis = 0)
            
            # Ersetzt Boolean Werte durch Strings
            df_tablematch[column] = df_tablematch[column].replace(boolean_to_string) # Muss ausgeführt werden, da sonst str.upper() einen Fehler wirft.

            # Ändert Datentyp in objects
            if df_tablematch[column].dtype != object:
                df_tablematch[column] = df_tablematch[column].astype(object)
                df_tablematch[column] = df_tablematch[column].map(str) # Sonst werden Zahlen nicht richtig erkannt
            
            # Füllt Null mit leeren Strings auf
            df_tablematch[column].fillna('', inplace = True) # NA werden zu leeren Strings

            # Macht die Strings groß
            if not case_sensitive:
                df_tablematch[column] = df_tablematch[column].str.upper()

            # Teilt die Werte bei einem Komma auf
            df_tablematch[column] = df_tablematch[column].str.split(pat = rule_delimiter)
            df_tablematch = df_tablematch.explode(column)
            df_tablematch[column] = df_tablematch[column].str.strip()

            # Reset Index
            df_tablematch.reset_index(drop = True, inplace = True)

            # Aufbereitung der Regeln
            for index, row in df_tablematch.iterrows():
                if row[column] == '':
                    df_tablematch.loc[index,column] = None
                elif row[column] == 'BLANK':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"].isna()'
                elif row[column] == '!BLANK':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"].notna()'
                elif row[column][0:2] == '!=':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] != ' + row[column][2:]
                elif row[column][0:2] == '>=':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] >= ' + row[column][2:]
                elif row[column][0:2] == '<=':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] <= ' + row[column][2:]
                elif row[column][0] == '>':
                    print(type(row[column][1:]))
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] > ' + row[column][1:]
                elif row[column][0] == '<':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] < ' + row[column][1:]
                elif row[column][0:2] == '==':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] == ' + row[column][2:]
                elif row[column][0] == '=':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] == ' + row[column][1:]
                elif row[column][0:2] == '!*':
                    if row[column][-1] == '*':
                        df_tablematch.loc[index,column] = '~df_main_table["' + column + '"].str.contains("' + row[column][2:-1] +'")'
                    else:
                        df_tablematch.loc[index,column] = '~df_main_table["' + column + '"].str.endswith("' + row[column][2:] +'")'
                elif row[column][0] == '*':
                    if row[column][-1] == '*':
                        df_tablematch.loc[index,column] = 'df_main_table["' + column + '"].str.contains("' + row[column][1:-1] +'")'
                    else:
                        df_tablematch.loc[index,column] = 'df_main_table["' + column + '"].str.endswith("' + row[column][1:] +'")'
                elif row[column][-1] == '*':
                    if row[column][0] == '!':
                        df_tablematch.loc[index,column] = '~df_main_table["' + column + '"].str.startswith("' + row[column][1:-1] +'")'
                    else:
                        df_tablematch.loc[index,column] = 'df_main_table["' + column + '"].str.startswith("' + row[column][:-1] +'")'
                elif row[column][0] == '!':
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] != "' + row[column] +'"'
                else:
                    df_tablematch.loc[index,column] = 'df_main_table["' + column + '"] == "' + row[column] +'"'

        # Test Daten aufbereiten
        mask_main_table = df_main_table.applymap(type) != bool
        # print(mask_main_table)
        # df_main_table = df_main_table.where(mask_main_table, df_main_table.replace(boolean_to_string)) # TODO stattdessen, map auf der Column?
        
        for column in column_inner:
            df_main_table = df_main_table.where(mask_main_table[column], df_main_table[column].map(boolean_to_string), axis = 0)

            # Ersetzt Boolean Werte durch Strings
            # df_main_table[column] = df_main_table[column].replace(boolean_to_string)

            # Ändert Datentyp in objects
            if df_main_table[column].dtype == bool:
                df_main_table[column] = df_main_table[column].astype(object)
            
            # Macht die Strings groß
            if not case_sensitive and df_main_table[column].dtype == object:
                df_main_table[column] = df_main_table[column].str.upper()

        # Initialisieren Ergebnisspalte
        df_main_table[tm_result] = None

        # Statement erstellen und ausführen
        for index, row in df_tablematch.iterrows():
            row_clean = row.dropna()
            
            row_clean_columns = row_clean.index.to_list()
            row_clean_columns.remove(tm_result)

            query_start = "df_main_table.loc[(df_main_table['" + tm_result + "'].isna())"
            query_end = ", '" + tm_result + "'] = '" + str(row_clean[tm_result]) + "'" # TODO anderer Dtype als String?

            query_mid = ""
            for row_column in row_clean_columns:
                query_mid = query_mid + " & (" + row_clean[row_column] + ")"

            query_full = query_start + query_mid + query_end # TODO Cache / Preprocess
            
            print(query_full)
            exec(query_full) # TODO Risiko für Code Injektion?
        
        if inplace:
            df_main_table_input[tm_result] = df_main_table[tm_result]
            return df_main_table_input
        else:
            output = df_main_table_input.copy()
            output[tm_result] = df_main_table[tm_result]
            return output

        # TODO Probleme bei dem Vergleich von Float mit Int?


In [167]:
# Test Case Int
Input = pd.DataFrame([
    [1],
    [2],
    [3],
    [4],
    [5],
    [6]], 
    columns = ['Test Number'])
# print(Input)

Rules = pd.DataFrame([
    ['>5', 'Greater than 5'],
    ['<5', 'Smaller than 5'],
    ['>=5', 'Equal, Greater than 5'],
    ['<=5', 'Equal, Smaller than 5'],
    ['==5', 'Eqauls 5'],
    ['!=5', 'Not Equals 5'],
    [None, 'Fallback']], 
    columns = ['Test Number', 'Output'])
# print(Rules)

IntTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",)
print(IntTest)

<class 'str'>
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] > 5), 'Output'] = 'Greater than 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] < 5), 'Output'] = 'Smaller than 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] >= 5), 'Output'] = 'Equal, Greater than 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] <= 5), 'Output'] = 'Equal, Smaller than 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] == 5), 'Output'] = 'Eqauls 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] != 5), 'Output'] = 'Not Equals 5'
df_main_table.loc[(df_main_table['Output'].isna()), 'Output'] = 'Fallback'
   Test Number                 Output
0            1         Smaller than 5
1            2         Smaller than 5
2            3         Smaller than 5
3            4         Smaller than 5
4  

In [168]:
# Test Case Float
Input = pd.DataFrame([
    [1.1],
    [2.3],
    [3.5],
    [4.6],
    [5.0],
    [6.1]], 
    columns = ['Test Number'])
# print(Input)

Rules = pd.DataFrame([
    ['>5', 'Greater than 5'],
    ['<5', 'Smaller than 5'],
    # ['>=5', 'Equal, Greater than 5'],
    # ['<=5', 'Equal, Smaller than 5'],
    ['==5', 'Eqauls 5'],
    ['!=5', 'Not Equals 5'],
    [None, 'Fallback']], 
    columns = ['Test Number', 'Output'])
# print(Rules)

FloatTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",)
print(FloatTest)

<class 'str'>
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] > 5), 'Output'] = 'Greater than 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] < 5), 'Output'] = 'Smaller than 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] == 5), 'Output'] = 'Eqauls 5'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test Number"] != 5), 'Output'] = 'Not Equals 5'
df_main_table.loc[(df_main_table['Output'].isna()), 'Output'] = 'Fallback'
   Test Number          Output
0          1.1  Smaller than 5
1          2.3  Smaller than 5
2          3.5  Smaller than 5
3          4.6  Smaller than 5
4          5.0        Eqauls 5
5          6.1  Greater than 5


In [169]:
# Test Case String
Input = pd.DataFrame([
    ['Hey Hallo'],
    ['Hallo Hey'],
    ['Hey Hallo Hey'],
    ['Hey Ciao'],
    ['Ciao Hey'],
    ['Hey Ciao Hey']], 
    columns = ['Test String'])
# print(Input)

Rules = pd.DataFrame([
    ['*Hallo', 'Ends with *Hallo'],
    ['Hallo*', 'Starts with Hallo*'],
    ['*Hallo*', 'Contains *Hallo*'],
    ['!*Hallo', 'Not Ends with !*Hallo'],
    ['!Hallo*', 'Not Starts with !Hallo*'],
    ['!*Hallo*', 'Not Contains !*Hallo*'],
    [None, 'Fallback']], 
    columns = ['Test String', 'Output'])
# print(Rules)

StringTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",)
print(StringTest)

StringTest2 = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",
    case_sensitive = False)
print(StringTest2)

df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.endswith("Hallo")), 'Output'] = 'Ends with *Hallo'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.startswith("Hallo")), 'Output'] = 'Starts with Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.contains("Hallo")), 'Output'] = 'Contains *Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.endswith("Hallo")), 'Output'] = 'Not Ends with !*Hallo'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.startswith("Hallo")), 'Output'] = 'Not Starts with !Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.contains("Hallo")), 'Output'] = 'Not Contains !*Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()), 'Output'] = 'Fallback'
     Test String                 Output
0      Hey Hallo       Ends wit

In [170]:
# Test Case Boolean
Input = pd.DataFrame([
    [True],
    [False],
    ['True'],
    ['False'],
    ['TRUE'],
    ['FALSE']], 
    columns = ['Boolean String'])
# print(Input)

Rules = pd.DataFrame([
    [True, 'True'],
    [False, 'False'],
    [None, 'Fallback']], 
    columns = ['Boolean String', 'Output'])
# print(Rules)

BooleanTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",)
print(BooleanTest)

BooleanTest2 = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",
    case_sensitive = False)
print(BooleanTest2)

df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] == "True"), 'Output'] = 'True'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] == "False"), 'Output'] = 'False'
df_main_table.loc[(df_main_table['Output'].isna()), 'Output'] = 'Fallback'
  Boolean String    Output
0           True      True
1          False     False
2           True      True
3          False     False
4           TRUE  Fallback
5          FALSE  Fallback
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] == "TRUE"), 'Output'] = 'True'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] == "FALSE"), 'Output'] = 'False'
df_main_table.loc[(df_main_table['Output'].isna()), 'Output'] = 'Fallback'
  Boolean String Output
0           True   True
1          False  False
2           True   True
3          False  False
4           TRUE   True
5          FALSE  False


In [171]:
# Test Case Blank
Input = pd.DataFrame([
    [],
    [True],
    [False],
    ['True'],
    ['False']], 
    columns = ['Boolean String'])
# print(Input)

Rules = pd.DataFrame([
    ['BLANK', 'BLANK'],
    ['!BLANK', 'Not BLANK'],
    [None, 'Fallback']], 
    columns = ['Boolean String', 'Output'])
# print(Rules)

BooleanTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",)
print(BooleanTest)

BooleanTest2 = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",
    case_sensitive = False)
print(BooleanTest2)

df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"].isna()), 'Output'] = 'BLANK'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"].notna()), 'Output'] = 'Not BLANK'
df_main_table.loc[(df_main_table['Output'].isna()), 'Output'] = 'Fallback'
  Boolean String     Output
0           None      BLANK
1           True  Not BLANK
2          False  Not BLANK
3           True  Not BLANK
4          False  Not BLANK
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"].isna()), 'Output'] = 'BLANK'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"].notna()), 'Output'] = 'Not BLANK'
df_main_table.loc[(df_main_table['Output'].isna()), 'Output'] = 'Fallback'
  Boolean String     Output
0           None      BLANK
1           True  Not BLANK
2          False  Not BLANK
3           True  Not BLANK
4          False  Not BLANK


In [172]:
# Test Case Date
Input = pd.DataFrame([
    ['2021-11-01'],
    ['2021-10-30'],
    ['2021-10-31'],
    ['2021-10-31'],
    ['2021-10-31'],
    ['2021-10-31']], 
    columns = ['Boolean String'], dtype = 'datetime64[ns]')
# print(Input)

Rules = pd.DataFrame([
    ['!="2021-10-31"', 'Not 2021-10-31'],
    ["<'2021-10-31'", 'Smaller than 2021-10-31'],
    ['>="2021-10-31"', 'Equal, Greater than 2021-10-31'],
    ['<="2021-10-31"', 'Equal, Smaller than 2021-10-31'],
    ['=="2021-10-31"', 'Eqauls 2021-10-31'],
    ['>"2021-10-31"', 'Greater than 2021-10-31']], 
    columns = ['Boolean String', 'Output'])
# print(Rules)

DateTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",)
print(DateTest)

<class 'str'>
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] != "2021-10-31"), 'Output'] = 'Not 2021-10-31'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] < '2021-10-31'), 'Output'] = 'Smaller than 2021-10-31'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] >= "2021-10-31"), 'Output'] = 'Equal, Greater than 2021-10-31'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] <= "2021-10-31"), 'Output'] = 'Equal, Smaller than 2021-10-31'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] == "2021-10-31"), 'Output'] = 'Eqauls 2021-10-31'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Boolean String"] > "2021-10-31"), 'Output'] = 'Greater than 2021-10-31'
  Boolean String                          Output
0     2021-11-01                  Not 2021-10-31
1     2021-10-30                  Not 2

In [173]:
# Test Case Category
Input = pd.DataFrame([
    ['Hey Hallo'],
    ['Hallo Hey'],
    ['Hey Hallo Hey'],
    ['Hey Ciao'],
    ['Ciao Hey'],
    ['Hey Ciao Hey']], 
    columns = ['Test String'], dtype = 'category')
# print(Input)
print(Input.info())

Rules = pd.DataFrame([
    ['*Hallo', 'Ends with *Hallo'],
    ['Hallo*', 'Starts with Hallo*'],
    ['*Hallo*', 'Contains *Hallo*'],
    ['!*Hallo', 'Not Ends with !*Hallo'],
    ['!Hallo*', 'Not Starts with !Hallo*'],
    ['!*Hallo*', 'Not Contains !*Hallo*'],
    [None, 'Fallback']], 
    columns = ['Test String', 'Output'])
# print(Rules)

StringTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",)
print(StringTest)

StringTest2 = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",
    case_sensitive = False)
print(StringTest2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Test String  6 non-null      category
dtypes: category(1)
memory usage: 342.0 bytes
None
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.endswith("Hallo")), 'Output'] = 'Ends with *Hallo'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.startswith("Hallo")), 'Output'] = 'Starts with Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.contains("Hallo")), 'Output'] = 'Contains *Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.endswith("Hallo")), 'Output'] = 'Not Ends with !*Hallo'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.startswith("Hallo")), 'Output'] = 'Not Starts with !Hallo*'
df_main_ta

In [174]:
# Test Case multiple rules
Input = pd.DataFrame([
    ['Hey Hallo'],
    ['Hallo Hey'],
    ['Hey Hallo Hey'],
    ['Hey Ciao'],
    ['Ciao Hey'],
    ['Hey Ciao Hey']], 
    columns = ['Test String'])
# print(Input)

Rules = pd.DataFrame([
    ['Hal*|He*', 'Hal OR He'],
    ['Hallo*', 'Starts with Hallo*'],
    ['*Hallo*', 'Contains *Hallo*'],
    ['!*Hallo', 'Not Ends with !*Hallo'],
    ['!Hallo*', 'Not Starts with !Hallo*'],
    ['!*Hallo*', 'Not Contains !*Hallo*'],
    [None, 'Fallback']], 
    columns = ['Test String', 'Output'])
# print(Rules)

StringTest = tablematch(
    df_tablematch_input = Rules, 
    df_main_table_input = Input,
    tm_result = "Output",
    rule_delimiter='|')
print(StringTest)


df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.startswith("Hal")), 'Output'] = 'Hal OR He'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.startswith("He")), 'Output'] = 'Hal OR He'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.startswith("Hallo")), 'Output'] = 'Starts with Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (df_main_table["Test String"].str.contains("Hallo")), 'Output'] = 'Contains *Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.endswith("Hallo")), 'Output'] = 'Not Ends with !*Hallo'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.startswith("Hallo")), 'Output'] = 'Not Starts with !Hallo*'
df_main_table.loc[(df_main_table['Output'].isna()) & (~df_main_table["Test String"].str.contains("Hallo")), 'Output'] = 'Not Contains !*Hallo*'
df_main_table.loc[(df_m